In [ ]:
import time
import pandas as pd  # 데이터프레임 및 CSV 저장을 위해
from datetime import datetime, timedelta  # 파일명 생성용 + 날짜 반복용
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# ====================================================
# 0. URL 생성 설정 (oid/date만 여기서 바꿔 쓰면 됨)
# ====================================================
BASE_URL = "https://news.naver.com/main/list.naver?mode=LPOD&mid=sec"
OID_LIST = ["018", "009", "014", "015", "277"]

START_DATE = "20251201"
END_DATE = "20251219"


def generate_targets(oid_list, start_yyyymmdd, end_yyyymmdd):
    start_dt = datetime.strptime(start_yyyymmdd, "%Y%m%d")
    end_dt = datetime.strptime(end_yyyymmdd, "%Y%m%d")

    cur = start_dt
    while cur <= end_dt:
        date_str = cur.strftime("%Y%m%d")
        for oid in oid_list:
            yield oid, date_str, f"{BASE_URL}&oid={oid}&date={date_str}"
        cur += timedelta(days=1)


# ====================================================
# 1. 크롬 드라이버 설정 및 실행
# ====================================================
def set_chrome_driver():
    chrome_options = Options()
    # chrome_options.add_argument("--headless")  # 브라우저 창 없이 실행하려면 주석 해제
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    # 봇 탐지 방지용 User-Agent 설정
    chrome_options.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    )

    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=chrome_options
    )
    return driver


# ====================================================
# 2. 기사 상세 내용 추출 함수 (핵심 로직)
# ====================================================
def extract_article_info(driver, url):
    try:
        driver.get(url)
        time.sleep(0.5)  # 페이지 로딩 및 리다이렉트 대기

        # [필터링] 연예 기사(entertain.naver.com) 여부 확인
        if "entertain.naver.com" in driver.current_url:
            print(f" -> [패스] 연예 기사로 확인되어 건너뜁니다.")
            return None

        # [대기] 제목 요소가 뜰 때까지 최대 3초 대기
        try:
            WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "#title_area > span"))
            )
        except:
            if "entertain.naver.com" in driver.current_url:
                print(f" -> [패스] 연예 기사(로딩 후 확인)로 건너뜁니다.")
                return None
            else:
                print(f" -> [스킵] 기사 형식이 다르거나 로딩 실패: {url}")
                return None

        # 1) 제목 추출
        try:
            title = driver.find_element(By.CSS_SELECTOR, "#title_area > span").text
        except:
            title = "제목 없음"

        # 2) 본문 추출 (이미지 캡션, 요약봇 등 불필요 요소 제거)
        try:
            dic_area = driver.find_element(By.CSS_SELECTOR, "#dic_area")

            driver.execute_script("""
                var element = arguments[0];
                var dirts = element.querySelectorAll(".img_desc, .media_end_summary");
                for (var i = 0; i < dirts.length; i++) {
                    dirts[i].remove();
                }
            """, dic_area)

            content = dic_area.text
            content = content.replace("\n", " ").strip()
        except:
            content = "본문 없음"

        # 3) 게재 시간 추출
        try:
            date_element = driver.find_element(
                By.CSS_SELECTOR,
                "#ct > div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span"
            )
            date_text = date_element.text
        except:
            try:
                date_text = driver.find_element(By.CSS_SELECTOR, ".media_end_head_info_datestamp span").text
            except:
                date_text = "날짜 없음"

        print(f"[수집 성공] {title} ({date_text})")

        return {
            "제목": title,
            "본문": content,
            "날짜": date_text,
            "링크": url
        }

    except Exception as e:
        print(f"[에러] 상세 페이지 수집 중 오류: {e}")
        return None


# ====================================================
# 3. 메인 실행 함수 (URL 주입 방식으로 변경)
# ====================================================
def main(target_url):
    driver = set_chrome_driver()
    driver.get(target_url)
    time.sleep(2)

    collected_data = []
    page_count = 1

    try:
        while True:
            print(f"\n===== [ {page_count} 페이지 탐색 중 ] =====")

            # --- [Step A] 현재 리스트 페이지의 기사 URL 수집 ---
            article_urls = []

            selectors = [
                "#main_content > div.list_body.newsflash_body > ul.type06_headline > li dl > dt:not(.photo) > a",
                "#main_content > div.list_body.newsflash_body > ul.type06 > li dl > dt:not(.photo) > a"
            ]

            for sel in selectors:
                links = driver.find_elements(By.CSS_SELECTOR, sel)
                for link in links:
                    url = link.get_attribute("href")
                    if url:
                        article_urls.append(url)

            print(f"-> 발견된 기사 링크: {len(article_urls)}개")

            # --- [Step B] 각 기사 상세 페이지 방문 및 데이터 추출 ---
            list_window = driver.current_window_handle

            for url in article_urls:
                if "entertain.naver.com" in url:
                    continue

                driver.execute_script("window.open('');")
                driver.switch_to.window(driver.window_handles[-1])

                data = extract_article_info(driver, url)
                if data:
                    collected_data.append(data)

                driver.close()
                driver.switch_to.window(list_window)

            # --- [Step C] 페이징(다음 페이지 이동) 처리 ---
            try:
                paging_area = driver.find_element(By.CSS_SELECTOR, "#main_content > div.paging")

                current_page_elem = paging_area.find_element(By.TAG_NAME, "strong")
                current_page_num = int(current_page_elem.text)
                next_page_num = current_page_num + 1

                next_page_btn = None
                try:
                    next_page_btn = paging_area.find_element(By.XPATH, f".//a[text()='{next_page_num}']")
                except:
                    next_page_btn = None

                if next_page_btn:
                    print(f"-> {next_page_num} 페이지로 이동합니다.")
                    next_page_btn.click()
                    page_count += 1
                    time.sleep(2)
                    continue

                try:
                    next_group_btn = paging_area.find_element(By.CSS_SELECTOR, "a.next")
                    print("-> '다음' 그룹(Next) 버튼을 클릭합니다.")
                    next_group_btn.click()
                    page_count += 1
                    time.sleep(2)
                    continue
                except:
                    print("-> 더 이상 다음 페이지가 없습니다. 수집을 종료합니다.")
                    break

            except Exception as e:
                print(f"-> 페이징 처리 중 에러 혹은 마지막 페이지 도달: {e}")
                break

    except KeyboardInterrupt:
        print("\n[사용자 중단] 작업을 강제로 중단합니다.")

    finally:
        driver.quit()

        # ====================================================
        # 4. 결과 저장 (CSV) - target_url 기준으로 저장
        # ====================================================
        print("\n[저장 시작] 데이터를 CSV 파일로 저장합니다...")

        if collected_data:
            df = pd.DataFrame(collected_data)

            # 파일명 생성 로직 (URL의 oid, date 파라미터 활용)
            try:
                params = target_url.split('?')[1]
                oid = [p for p in params.split('&') if 'oid=' in p][0]
                date_val = [p for p in params.split('&') if 'date=' in p][0]
                filename = f"{oid}&{date_val}.csv"
            except:
                now = datetime.now().strftime("%Y%m%d_%H%M%S")
                filename = f"naver_news_result_{now}.csv"

            df.to_csv(filename, index=False, encoding='utf-8-sig')

            print(f"✅ 저장 완료! 파일명: {filename}")
            print(f"📊 총 {len(df)}개의 기사가 수집되었습니다.")
        else:
            print("❌ 수집된 데이터가 없습니다.")


# ====================================================
# 5. 반복 실행 진입점 (oid/date 루프)
# ====================================================
if __name__ == "__main__":
    for oid, date_str, url in generate_targets(OID_LIST, START_DATE, END_DATE):
        print(f"\n\n==============================")
        print(f"RUN -> oid={oid}, date={date_str}")
        print(f"URL -> {url}")
        print(f"==============================\n")
        main(url)




RUN -> oid=018, date=20251201
URL -> https://news.naver.com/main/list.naver?mode=LPOD&mid=sec&oid=018&date=20251201


===== [ 1 페이지 탐색 중 ] =====
-> 발견된 기사 링크: 20개
 -> [스킵] 기사 형식이 다르거나 로딩 실패: https://n.news.naver.com/mnews/article/018/0006175549
[수집 성공] "그 사람 車 아닌데"...1억 BMW '활활', 알고보니 (2025.12.01. 오후 11:03)
[수집 성공] ‘청산 위기’ 홈플러스에…노조, 물·소금 끊고 아사 단식 돌입 (2025.12.01. 오후 10:58)
[수집 성공] 폐지 줍는 할머니에 ‘무제한 카드’ 준 유튜버…"85년 살며 처음" (2025.12.01. 오후 10:48)
[수집 성공] 또 자택서 쓰러진 ‘쿠팡 새벽배송’ 기사…노조 측, 진상규명 촉구 (2025.12.01. 오후 10:44)
[수집 성공] 조건만남 응한 남성에게 강도질 지시…10대 구속 (2025.12.01. 오후 10:31)
[수집 성공] `민중기 특검`의 강압 수사, 故 양평군 공무원 유서에 써 있었다(종합) (2025.12.01. 오후 10:28)
[수집 성공] '음주운전 3회 시 가중처벌' 위헌 논란…헌재 "만장일치 합헌" (2025.12.01. 오후 10:15)
[수집 성공] 기술혁신 3대 학회, 바이오·AI 대응방안 논의 (2025.12.01. 오후 9:57)
[수집 성공] 홍대입구역 승강장서 화재…역무원 자체 진화로 인명피해 없어 (2025.12.01. 오후 9:45)
[수집 성공] "한국 여행 즐거웠습니다"…모금함에 110만원 남긴 日여행객 (2025.12.01. 오후 9:41)
[수집 성공] '로봇위성 전문' 워커린스페이스, 90억원 규모 프리A 투자 유치 (2025.12.01. 오후 9:40)
[수집 성공] ‘눈물’의 엔비디아 주식 매각…손정의 “한주도 팔고 싶

[수집 성공] [인사]금융위원회 (2025.12.01. 오후 4:54)
[수집 성공] 인피니트헬스케어 "주총 개최금지 등 가처분 소송 제기" (2025.12.01. 오후 4:53)
[수집 성공] 뒤늦게 등록했지만…옥주현, '기획사 미등록' 여파로 검찰 송치 (2025.12.01. 오후 4:53)
[수집 성공] "3조원 주파수 재할당, '경매가 기준' 뜯어고쳐야"…전파법 개정 요구도 (2025.12.01. 오후 4:53)
[수집 성공] [단독]“2년 전부터 준비”…솔루엠, ESL 분사 후 상폐까지 고려 (2025.12.01. 오후 4:51)
[수집 성공] 삼영엠텍, 700억 규모 종속사 주식 취득 결정 (2025.12.01. 오후 4:50)
 -> [스킵] 기사 형식이 다르거나 로딩 실패: https://n.news.naver.com/mnews/article/018/0006175415
[수집 성공] 쿼드메디슨, 공모가 ‘상단’ 1만 5000원 확정…2~3일 청약 (2025.12.01. 오후 4:47)
[수집 성공] 엑시콘, 66.6억 규모 자사주 처분 결정 (2025.12.01. 오후 4:46)
[수집 성공] 한국GM, 11월 총 4만3799대 판매…전년 대비 11.7% 감소 (2025.12.01. 오후 4:46)
 -> [패스] 연예 기사로 확인되어 건너뜁니다.
[수집 성공] 페스카로·이지스·쿼드메디슨까지…IPO 흥행 속 동시 청약 돌입 (2025.12.01. 오후 4:45)
[수집 성공] 아이진 “대주주 한국비엠아이, 유증 120% 초과청약 참여” (2025.12.01. 오후 4:44)
[수집 성공] 덕산테코피아, 김병기 대표이사로 변경 (2025.12.01. 오후 4:44)
-> 8 페이지로 이동합니다.

===== [ 8 페이지 탐색 중 ] =====
-> 발견된 기사 링크: 20개
[수집 성공] “코스피 조정 끝물”…12월 ‘산타랠리’ 올까 (2025.12.01. 오후 4:44)
[수집 성공] 우리은행, 중국·인도네시아 출국 전 미리 계좌 개설 (202

[수집 성공] 최대호 안양시장, 입양아 후견인이 되다 (2025.12.01. 오후 3:07)
[수집 성공] 박광선 아주산업 레미콘사업본부장, 산업통상부 장관 표창 수상 (2025.12.01. 오후 3:06)
[수집 성공] 릴리이브, ‘그로우턴 엑소좀 브러쉬 앰플’ 출시… 성분·흡수력·편의성 전면 업그레이드 (2025.12.01. 오후 3:05)
[수집 성공] "무리한 짜맞추기 기소"…오세훈 시장, 김건희 특검 기소에 반발 (2025.12.01. 오후 3:05)
-> 14 페이지로 이동합니다.

===== [ 14 페이지 탐색 중 ] =====
-> 발견된 기사 링크: 20개
[수집 성공] 로스쿨생 '학업스트레스' 디지털 심리케어 지원한다 (2025.12.01. 오후 3:04)
[수집 성공] 오상헬스케어, 이동현 회장 '금탑산업훈장' 수훈 (2025.12.01. 오후 3:03)
[수집 성공] 그린리소스, 44.8억 규모 공급계약 체결 (2025.12.01. 오후 3:03)
[수집 성공] 11살 된 원·위안화 직거래 시장…“위안화 무역결제 두 배 증가할 것” (2025.12.01. 오후 3:02)
[수집 성공] 野박수영 “민주당 반도체특별법, 52시간 예외 빠진 ‘빈껍데기’” (2025.12.01. 오후 3:02)
[수집 성공] 쿠팡 무슨 수로 뚫었나 봤더니…고객정보 문 열려있었다 (2025.12.01. 오후 3:02)
[수집 성공] 부스터즈, 200억 규모 상환전환우선주 발행 결정 (2025.12.01. 오후 3:01)
[수집 성공] 공정위 부위원장 "상생, 기업 경쟁력 높이고 생태계 건강하게 해" (2025.12.01. 오후 3:01)
[수집 성공] 금감원 '삼성생명 일탈회계' 중단 결정…이찬진 "늦어도 1월 말 정리" (2025.12.01. 오후 3:01)
[수집 성공] 정영선 조경사 등 8명 '2025 삼성행복대상' 수상 (2025.12.01. 오후 3:01)
[수집 성공] 이찬진 “대출 절벽 없다…ELS 제재에도 생산적 금융 위축 막을 것” (2025.1

 -> [패스] 연예 기사로 확인되어 건너뜁니다.
 -> [패스] 연예 기사로 확인되어 건너뜁니다.
[수집 성공] 호반그룹 '오너 2세' 김민성 부사장 승진…"중장기 전략 수립 중책" (2025.12.01. 오후 12:08)
[수집 성공] 경찰, ‘성추행 피소’ 장경태 수사…관련 자료 등 확보 (2025.12.01. 오후 12:04)
[수집 성공] `서부지법 난동` 사랑제일교회 특임전도사, 2심서 징역 3년 6개월 (2025.12.01. 오후 12:03)
 -> [패스] 연예 기사로 확인되어 건너뜁니다.
[수집 성공] 피치, 한국 연례협의 실시…경제상황·전망 등 점검 (2025.12.01. 오후 12:02)
[수집 성공] 10월 온라인쇼핑거래액 22.7조…"소비쿠폰·테슬라가 이끌었다" (2025.12.01. 오후 12:02)
[수집 성공] ‘쿠팡 개인정보 유출’ 中 직원 소행?…경찰, 수사 나섰다 (2025.12.01. 오후 12:02)
[수집 성공] 겨울 한파 대비 ‘한랭질환 감시’ 시작…고령층 비중 30% (2025.12.01. 오후 12:01)
[수집 성공] 美증시 호조 지속에 기관 해외투자 역대 최대…증가세는 ‘주춤’ (2025.12.01. 오후 12:01)
[수집 성공] 소방청, 소규모 지하주차장·리튬전지공장 소방시설 기준 강화 (2025.12.01. 오후 12:01)
[수집 성공] 공정위 하도급대금 지급명령 불이행…계성건설 檢 고발 (2025.12.01. 오후 12:01)
[수집 성공] 라이엇 게임즈, 6일부터 LCK 연말 사진전 (2025.12.01. 오후 12:01)
[수집 성공] "앞으로 집·직장 인근서 운전학원 도로 연수 받을 수 있어" (2025.12.01. 오후 12:00)
[수집 성공] 한전KPS, 디지털경영혁신대상 부총리상 수상 (2025.12.01. 오전 11:53)
-> '다음' 그룹(Next) 버튼을 클릭합니다.

===== [ 21 페이지 탐색 중 ] =====
-> 발견된 기사 링크: 20개
[수집 성공] ‘쿠팡 개인정보 유출

[수집 성공] GH, 지방공기업 최초 '내부회계관리제도' 도입 (2025.12.01. 오전 10:15)
[수집 성공] '젊은 도시' 의왕 고천 ‘의왕시청역 SK뷰 아이파크’ 12월 1일 특별공급 (2025.12.01. 오전 10:14)
[수집 성공] 중고생까지 강제가입..1200억대 도박사이트 조직 검거 (2025.12.01. 오전 10:13)
-> 27 페이지로 이동합니다.

===== [ 27 페이지 탐색 중 ] =====
-> 발견된 기사 링크: 20개
[수집 성공] "수소 산업의 모든 것 집약"…'WHE 2025' 개최 D-3 (2025.12.01. 오전 10:13)
[수집 성공] 3380만명 털린 '쿠팡 개인정보 유출' 집단소송 간다 (2025.12.01. 오전 10:11)
[수집 성공] "뽀로로 여권 들고 국가유산으로 여행 떠나요" (2025.12.01. 오전 10:10)
[수집 성공] 경차 성수기 하루 20만원..제주 렌터카 '바가지' 손보기로 (2025.12.01. 오전 10:10)
 -> [패스] 연예 기사로 확인되어 건너뜁니다.
[수집 성공] 파월 후임은 해싯?…트럼프 “연준 의장 지명자 결정했다” (2025.12.01. 오전 10:08)
 -> [스킵] 기사 형식이 다르거나 로딩 실패: https://n.news.naver.com/mnews/article/018/0006175021
[수집 성공] 도프, 쥬브아셀로 메디컬 에스테틱 진화...‘피부 안정·기능 회복 모두 챙겨’ (2025.12.01. 오전 10:07)
[수집 성공] 로봇 위소매절제술, 합병증 낮춰, 고도비만 수술 판도 바꿔 (2025.12.01. 오전 10:06)
[수집 성공] 씨씨에스 "임시주총 개최금지 가처분 관련 항고 제기" (2025.12.01. 오전 10:05)
[수집 성공] DL이앤씨·삼성물산, 1.9조 규모 증산4구역 시공 맡는다 (2025.12.01. 오전 10:04)
[수집 성공] "아이 데리고 집 나가라"…재혼한 전처의 소송 폭탄 (2025.12.01. 오전 1

[수집 성공] [속보]11월 수출액 610억달러…전년대비 8.4% 증가 (2025.12.01. 오전 9:01)
[수집 성공] 11번가, 파리바게뜨·던킨 등 e쿠폰 특가 판매 (2025.12.01. 오전 9:01)
[수집 성공] '계엄 1년' 경찰 대국민 사과…"위헌·위법 행위 절대 협조 않을 것" (2025.12.01. 오전 9:01)
[수집 성공] 술·담배 살 때 신분증·결제 한 번에…통신3사 ‘PASS 신분증결제’ 시작 (2025.12.01. 오전 9:01)
[수집 성공] LG유플러스 “상품·사업 조직 분리로 AX 성과 본격화”…조직개편 (2025.12.01. 오전 9:01)
[수집 성공] 이노션, ‘대한민국광고대상’ 대상 8개…역대 최고 성적 (2025.12.01. 오전 9:01)
[수집 성공] 내년부터 증권거래세율 0.15%→0.20% 적융 (2025.12.01. 오전 9:00)
[수집 성공] 상권데이터 기반 ‘AI 창업·경영 컨설턴트’, 10대 민생 프로젝트 선정 (2025.12.01. 오전 9:00)
[수집 성공] “연금계좌 맞춤 포트폴리오부터 비과세 증여까지 한번에” (2025.12.01. 오전 8:58)
[수집 성공] CJ제일제당, CJ더마켓서 ‘쿵야 레스토랑즈’와 특별한 협업 (2025.12.01. 오전 8:58)
[수집 성공] 에스넷시스템, ‘2025 AIoT 국제전시회’에서 AI 모니터링 시스템 ‘IMS’ 공개 (2025.12.01. 오전 8:58)
 -> [패스] 연예 기사로 확인되어 건너뜁니다.
[수집 성공] AIA생명, 보험사기 의심 사례 적발…OCR 기술 고도화 결과 (2025.12.01. 오전 8:57)
[수집 성공] 한컴, ‘한컴오피스 AI 패키지’ GS인증 1등급 획득 (2025.12.01. 오전 8:55)
[수집 성공] 5일부터 ‘유니클로 감사제’ 돌입…주요 상품 특가에 (2025.12.01. 오전 8:53)
[수집 성공] LCC도 겨울철 공항 '코트룸' 서비스 잇단 개시 (2025.12.01. 오전 8:52)
[수집 성공] 

[수집 성공] K치킨의 오래된 미래…젠슨 황이 두번 찾은 '홀매장'[생생확대경] (2025.12.01. 오전 6:03)
[수집 성공] [오늘의 운세] 2025년 12월 1일 별자리 운세 (2025.12.01. 오전 6:03)
[수집 성공] [오늘의 운세] 2025년 12월 1일 띠별 운세 (2025.12.01. 오전 6:02)
[수집 성공] 이젠 패딩도 편의점에서 산다?…‘간편복’ 인기 심상찮네 (2025.12.01. 오전 6:02)
[수집 성공] 바가지 없애고 개막식 간소화했더니…"지역축제 관람객 2배 증가" (2025.12.01. 오전 6:01)
[수집 성공] 산은, 차명계좌 79억 세금 환불 소송서 패소…대법 파기환송 (2025.12.01. 오전 6:01)
[수집 성공] “마지막 할인 축제” 롯데아울렛, ‘더 블랙위크’ 진행 (2025.12.01. 오전 6:01)
[수집 성공] 자산운용사 3분기 순이익 9447억원…전년比 128% 급증 (2025.12.01. 오전 6:00)
-> 40 페이지로 이동합니다.

===== [ 40 페이지 탐색 중 ] =====
-> 발견된 기사 링크: 20개
[수집 성공] 막바지 향하는 특검 수사…법원으로 쏟아지는 재판 (2025.12.01. 오전 5:50)
[수집 성공] "친환경은 물론 층간소음도 제로…모듈러 건축, 학교·빌딩·아파트도 가능하죠" (2025.12.01. 오전 5:46)
[수집 성공] 12·3 계엄 1년…윤석열·김건희 재판 선고 내년 1~2월 나온다 (2025.12.01. 오전 5:46)
[수집 성공] 3대 특검, 尹부부 기소 성과…잇단 영장 기각 비판 목소리도 (2025.12.01. 오전 5:40)
[수집 성공] 강성 지지층에 갇힌 與野…민주주의 위기 겪고도 '심화된 양극화' (2025.12.01. 오전 5:35)
[수집 성공] “휴무일 없어도 돼”…새벽배송 ‘월 581만원’ 생계 논란 (2025.12.01. 오전 5:34)
[수집 성공] 국내 항공사, SAF 시범 도입 확대…"정부 지원 필요" (2025.